# **Security Course**
### <span style="font-weight:bold;">Instructor:</span> <span style="font-size:19px;"><a href="https://ir.linkedin.com/in/hrshahriari">Dr. Hamid Reza Shahriari</a></span>
### <span style="font-weight:bold;">Orgagnization</span>
### <span style="font-size:19px;"><a href="https://aut.ac.ir/en">Amirkabir University of Technology (Tehran Polytechnic)</a></span>
<img src="../assets/AKUT-logo.png" alt="Amirkabir University of Technology logo" style="width:300px;">

# **DES Encryption Algorithm**
In this notebook, we will walk through the steps of implementing the Data Encryption Standard (DES) algorithm.

### imports

In [ ]:
import utils
import sboxes

### key scheduler

In [ ]:
def key_scheduler(key: str)-> list[str]:
    """_summary_

    Args:
        key (str): input key

    Returns:
        list[str]: list of the subkeys for each round
    """
    pass

### round function

In [ ]:
def round_fucntion(R: str, round_key: str) -> str:
    """
    _summary_

    Args:
        R (str): lowest 32 bits of round input
        round_key (str): subkey of the round

    Returns:
        str: round output
    """
    pass

### DES algorithm

In [ ]:
def DES(plain_text: str, key: str) -> str:
    pass